In [2]:
import pandas as pd

# Charger les données brutes
df = pd.read_csv("Electric_Vehicle_Population_Data.csv")

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150482 entries, 0 to 150481
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         150482 non-null  object 
 1   County                                             150479 non-null  object 
 2   City                                               150479 non-null  object 
 3   State                                              150482 non-null  object 
 4   Postal Code                                        150479 non-null  float64
 5   Model Year                                         150482 non-null  int64  
 6   Make                                               150482 non-null  object 
 7   Model                                              150482 non-null  object 
 8   Electric Vehicle Type                              150482 non-null  object

In [3]:
df_clean = df.dropna(subset=["Model Year"])  # retire lignes sans année

df_clean["Model Year"] = df_clean["Model Year"].astype(int)


In [4]:
df_year = df_clean.groupby("Model Year")["VIN (1-10)"].count().reset_index()
df_year.columns = ["ds", "y"]  # Prophet impose ds/y
df_year["ds"] = pd.to_datetime(df_year["ds"], format="%Y")

df_year


,ds,y
0,1997-01-01,1
1,1998-01-01,1
2,1999-01-01,4
3,2000-01-01,8
4,2002-01-01,2
5,2003-01-01,1
6,2008-01-01,19
7,2010-01-01,24
8,2011-01-01,796
9,2012-01-01,1633


In [ ]:
from prophet import Prophet

model = Prophet(
    yearly_seasonality=False,
    changepoint_prior_scale=0.3
)

model.fit(df_year)

future = model.make_future_dataframe(periods=6, freq='Y')
forecast = model.predict(future)


In [ ]:
model.plot(forecast)
model.plot_components(forecast)

# Export pour Power BI
forecast_export = forecast[["ds","yhat"]]
forecast_export.columns = ["Year", "Forecast_Total_Vehicles"]

forecast_export.to_csv("Forecast_EV_US_2030.csv", index=False)
